In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('lr').getOrCreate()

24/05/06 14:57:22 WARN Utils: Your hostname, willians-desktop resolves to a loopback address: 127.0.1.1; using 192.168.1.73 instead (on interface enp1s0)
24/05/06 14:57:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/05/06 14:57:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [23]:
df = spark.read.csv('Mumbai.csv', inferSchema=True, header=True)

In [67]:
df = df.drop('Location').na.fill(0.0)

In [68]:
df.columns

['Price',
 'Area',
 'No. of Bedrooms',
 'Resale',
 'MaintenanceStaff',
 'Gymnasium',
 'SwimmingPool',
 'LandscapedGardens',
 'JoggingTrack',
 'RainWaterHarvesting',
 'IndoorGames',
 'ShoppingMall',
 'Intercom',
 'SportsFacility',
 'ATM',
 'ClubHouse',
 'School',
 '24X7Security',
 'PowerBackup',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'Hospital',
 'WashingMachine',
 'Gasconnection',
 'AC',
 'Wifi',
 "Children'splayarea",
 'LiftAvailable',
 'BED',
 'VaastuCompliant',
 'Microwave',
 'GolfCourse',
 'TV',
 'DiningTable',
 'Sofa',
 'Wardrobe',
 'Refrigerator']

In [26]:
feature_columns = ['Price',
 'Area',
 'Resale',
 'Gymnasium',
 'SwimmingPool',
 'LandscapedGardens',
 'JoggingTrack',
 'IndoorGames',
 'ShoppingMall',
 'Intercom',
 'SportsFacility',
 'ATM',
 'ClubHouse',
 'School',
 'CarParking',
 'StaffQuarter',
 'Cafeteria',
 'MultipurposeRoom',
 'Hospital',
 'WashingMachine',
 'Gasconnection',
 'AC',
 "Children'splayarea",
 'BED',
 'VaastuCompliant',
 'Microwave',
 'GolfCourse',
 'TV',
 'DiningTable',
 'Sofa',
 'Wardrobe',
 'Refrigerator']

In [69]:
assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')

In [70]:
output = assembler.transform(df)

In [71]:
data = output.withColumn('label', output['Price']).select(['features', 'label'])

In [72]:
train, test = data.randomSplit([0.7, 0.3])

In [73]:
train.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|               5379|
|   mean|1.493843705056702E7|
| stddev|2.012638624875193E7|
|    min|            2000000|
|    max|          400000000|
+-------+-------------------+



In [74]:
test.describe().show()

+-------+--------------------+
|summary|               label|
+-------+--------------------+
|  count|                2340|
|   mean|1.5344889722649572E7|
| stddev| 2.140225870506035E7|
|    min|             2000000|
|    max|           420000000|
+-------+--------------------+



In [75]:
lr = LinearRegression(featuresCol='features', labelCol='label', regParam=0.3)

In [76]:
lrModel = lr.fit(train)

In [77]:
result = lrModel.evaluate(test)

In [81]:
result.r2

0.9999999999999998

In [82]:
results = lrModel.transform(test)

In [83]:
results.show()

+--------------------+--------+--------------------+
|            features|   label|          prediction|
+--------------------+--------+--------------------+
|(32,[0,1],[200000...| 2000000|  2000000.2090302194|
|(32,[0,1],[201800...| 2018000|  2018000.2003751616|
|(32,[0,1],[210000...| 2100000|   2100000.227523423|
|(32,[0,1],[235000...| 2350000|   2350000.252151407|
|(32,[0,1],[1.32E7...|13200000|1.3200000221208118E7|
|(32,[0,1,2],[3800...| 3800000|   3800000.133666132|
|(32,[0,1,2],[4400...| 4400000|    4400000.18361202|
|(32,[0,1,2],[6000...| 6000000|   6000000.206148709|
|(32,[0,1,2],[7000...| 7000000|   7000000.160194424|
|(32,[0,1,2],[7000...| 7000000|   7000000.191721071|
|(32,[0,1,2],[8500...| 8500000|   8500000.083752237|
|(32,[0,1,2],[9000...| 9000000|   9000000.055304296|
|(32,[0,1,2],[9000...| 9000000|   9000000.084976433|
|(32,[0,1,2,3,4,5,...| 8000000|   7999999.926400189|
|(32,[0,1,2,3,4,5,...| 3800000|   3800000.050484453|
|(32,[0,1,2,3,4,5,...|11000000|1.1000000052126